### **APP DE CONEXIÓN BASE DE DATOS**

---

In [4]:
# Importación de librerias utilizadas para la conección y tratamiento de la información
# Conección
from sqlalchemy import create_engine
from sqlalchemy.engine import URL 
import pypyodbc as odbc
# Tratamiento de información
import pandas as pd
import pandasql as ps
import numpy as np
# Tratamiento de fechas
from datetime import date
# Usuarios y contraseñas protegidas
import getpass

In [5]:
# Solicitar usuario y contraseña de acceso
SERVER_NAME = input('NOMBRE DEL SERVIDOR: ')
DATABASE_NAME = input('NOMBRE DE DATABASE: ')
db = input('NOMBRE TABLA/VISTA: ')
UID = input('Ingresa tu usuario: ')
PWD = getpass.getpass('Ingresa tu contraseña: ')

In [6]:
# Variable de login
connection_string = f'''
Driver={{SQL Server}};
Server={SERVER_NAME};
Database={DATABASE_NAME};
Uid={UID};
Pwd={PWD};
'''

In [7]:
# Variable SQL: Consulta SQL para extracción de data
# Filtro de fecha: WHERE CONVERT(DATE, fecha, 103) >= CONVERT(DATE, GETDATE()-1, 103)
# Código 103 corresponde al formateo de fecha DD/MM/YYYY
sql_query = f'''

SELECT *
FROM {db}
-- Filtrar por registros cuya fecha sea la fecha máxima presente en la vista
WHERE fecha IN (SELECT MAX(fecha) FROM {db})
-- O por registros cuya fecha tenga el máximo valor de fecha para el mes anterior
   OR CONVERT(DATE, fecha) = (
       SELECT MAX(CONVERT(DATE, fecha))
       FROM {db}
       WHERE MONTH(fecha) = MONTH(DATEADD(MONTH, -1, GETDATE()))
   );

'''

In [8]:
# Script de conexión con base de datos y servidor
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})
engine = create_engine(connection_url, module=odbc)

In [9]:
# Script de descarga de data de la base de datos de acuerdo a la consulta SQL
df = pd.read_sql_query(sql_query, engine)

---

#### **ANÁLISIS DE DATA**

Funciones personalizadas

In [10]:
# Funciones de formateo de registros
# Formeteo de fecha
def date_formatting(fecha):
    date = fecha
    if type(date) == str:
        date = pd.to_datetime(date)
    result = date.strftime('%d-%m-%Y')
    return result

# Formateo de numeros
def num_formatting(numero):
    return '{:,.2f}'.format(numero)

# Proceso de DataFrame a travez de pandasql as ps para realizar consultas sql
def show_query(query, df):
    result = ps.sqldf(query, locals())
    return result

Datos generales del DataFrame "df"

In [11]:
# Variables de la base de datos
fecha_min = date_formatting(df['fecha'].min())
fecha_max = date_formatting(df['fecha'].max())
cuenta_df = num_formatting(df['fecha'].nunique())
cuenta_total_registros = num_formatting(df['fecha'].count())
cuenta_registros_df_max = num_formatting((df['fecha'] == df['fecha'].max()).sum())

In [12]:
# Visualización de información "df"
print("Datos de Generales")
print(f"Tabla: {db}")
print('------------------------------------------------------------')
print(f"Fecha mínima:       {fecha_min}")
print(f"Fecha máxima:       {fecha_max}")
print(f"Bases de datos:     {cuenta_df} Bases de datos")
print(f"Registos totales:   {cuenta_total_registros} en {cuenta_df} DB")
print(f"Registos ultima DB: {cuenta_registros_df_max} regristros de fecha {fecha_max}")

Datos de Generales
Tabla: vst_matriz_tbl
------------------------------------------------------------
Fecha mínima:       31-12-2023
Fecha máxima:       17-01-2024
Bases de datos:     2.00 Bases de datos
Registos totales:   31,882.00 en 2.00 DB
Registos ultima DB: 15,934.00 regristros de fecha 17-01-2024


In [13]:
# Datos generales de la tabla
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31882 entries, 0 to 31881
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   agencia          31882 non-null  int64         
 1   nombreagencia    31882 non-null  object        
 2   prestamo         31882 non-null  int64         
 3   fecha            31882 non-null  datetime64[ns]
 4   codigooficial    31882 non-null  int64         
 5   totalcartera     31882 non-null  float64       
 6   prevision        31880 non-null  float64       
 7   estado           31882 non-null  int64         
 8   moneda           31882 non-null  int64         
 9   fechaincumplido  6045 non-null   datetime64[ns]
 10  mora             6045 non-null   float64       
 11  oficial          31882 non-null  object        
 12  nrosocio         31882 non-null  int64         
 13  nombrecliente    31882 non-null  object        
 14  saldodiferido    8907 non-null   float

In [14]:
# Descripción general de las columnas de la tabla
print(df.describe())

            agencia      prestamo                          fecha  \
count  31882.000000  3.188200e+04                          31882   
mean      21.461326  1.001922e+07  2024-01-08 12:39:37.743076352   
min       10.000000  1.439000e+03            2023-12-31 00:00:00   
25%       12.000000  1.009491e+07            2023-12-31 00:00:00   
50%       18.000000  1.159026e+07            2023-12-31 00:00:00   
75%       28.000000  1.249236e+07  2024-01-17 01:30:02.836999936   
max       42.000000  1.429103e+07     2024-01-17 01:30:02.837000   
std        9.830126  4.334103e+06                            NaN   

       codigooficial   totalcartera      prevision        estado  \
count   31882.000000   31882.000000   31880.000000  31882.000000   
mean      496.044978    6848.112362     309.920250      2.272630   
min        41.000000       0.000000       0.000000      2.000000   
25%       428.000000    1337.336370       5.471210      2.000000   
50%       515.000000    2725.536443      52.287